# Financial App

In [6]:
import os
from pathlib import Path
import pandas as pd

# Get the current working directory
root_path = Path(os.getcwd())
if  root_path.name == "notebooks":
    print("✅ La última carpeta es 'notebooks'.")
    os.chdir(f"{root_path}/..")
    print(f"Now in directory: {os.getcwd()}")
else:
    print(f"Now in directory: {os.getcwd()}")

# Import your module
from app.modules.database.connection import engine
import app.modules.database.structur_databases as sdb
import app.modules.database.companies as companies
import app.modules.database.customers as customers
import app.modules.database.credit_manager as credit_manager
import app.modules.database.portfolio_manager as portfolio_manager
import app.modules.database.collection as colltions

from app.modules.database.credit_manager import credits_balance

from importlib import reload

reload(customers)
reload(companies)
reload(portfolio_manager)
reload(credit_manager)

%run logs/run_log.py

Now in directory: c:\Users\juanm_8qa8lav\Documents\Proyectos_Personales\FinancialApp
✅ The system correctly imports all modules.
✅ SQL script executed successfully.
✅ 24 provinces successfully inserted into the database.
✅ The company NeoCrediT S.A. has been added.
✅ New business plan created, its ID is 1 and it was associated with the company NeoCrediT S.A..
✅ The company Onoyen S.R.L. has been added.
✅ New business plan created, its ID is 2 and it was associated with the company Onoyen S.R.L..
✅ The company Asociación Mutual Union Federal has been added.
✅ New business plan created, its ID is 3 and it was associated with the company Asociación Mutual Union Federal.
✅ New business plan created, its ID is 4 and it was associated with the company Asociación Mutual Union Federal.
✅ New business plan created, its ID is 5 and it was associated with the company Asociación Mutual Union Federal.
✅ Settings successfully updated in the database.
✅ Supplier 2 and business plan 2 are valid and co

In [10]:
pp = pd.read_sql('portfolio_purchases', engine, index_col='ID')
customers = pd.read_sql('customers', engine, index_col='ID')
credits = pd.read_sql('credits', engine, index_col='ID')
collections = pd.read_sql('collection', engine, index_col='ID')
installments = pd.read_sql('installments', engine, index_col='ID')
companies = pd.read_sql('companies', engine, index_col='ID')
bp = pd.read_sql('business_plan', engine, index_col='ID')
companies = pd.read_sql('companies', engine, index_col='ID')

balance = credits_balance().copy()
print(f'Capital: $ {balance['Capital'].sum():,.2f}')
print(f'Interés: $ {balance['Interest'].sum():,.2f}')
print(f'IVA: $ {balance['IVA'].sum():,.2f}')
print(f'Total: $ {balance['Total'].sum():,.2f}')

# balance = balance.loc[(balance['Owner'] == 'NeoCrediT S.A.')].copy()
balance[['Capital', 'Interest', 'IVA', 'Total']] = balance[['Capital', 'Interest', 'IVA', 'Total']].map('${:,.2f}'.format)

balance

Capital: $ 43,723,832.79
Interés: $ 39,623,640.79
IVA: $ 8,320,964.45
Total: $ 91,668,438.51


,ID_Op,Nro_Inst,D_Due,Capital,Interest,IVA,Total,Anchorer,Owner
ID,,,,,,,,,
1,1,1,2022-10-20,$0.00,$0.00,$0.00,$0.00,Onoyen S.R.L.,NeoCrediT S.A.
2,1,2,2022-11-20,$0.00,$0.00,$0.00,$0.00,Onoyen S.R.L.,NeoCrediT S.A.
3,1,3,2022-12-20,$0.00,$0.00,$0.00,$0.00,Onoyen S.R.L.,NeoCrediT S.A.
4,1,4,2023-01-20,$0.00,$0.00,$0.00,$0.00,Onoyen S.R.L.,NeoCrediT S.A.
5,1,5,2023-02-20,$0.00,$0.00,$0.00,$0.00,Onoyen S.R.L.,NeoCrediT S.A.
...,...,...,...,...,...,...,...,...,...
5838,437,1,2025-02-17,$0.00,$0.00,$0.00,$0.00,NeoCrediT S.A.,NeoCrediT S.A.
5839,438,1,2025-02-26,$0.00,$0.00,$0.00,$0.00,NeoCrediT S.A.,NeoCrediT S.A.
5840,439,1,2025-03-07,$0.00,$0.00,$0.00,$0.00,NeoCrediT S.A.,NeoCrediT S.A.
